# MoveIt! Tutorial

## 1. Setup

The interface with MoveIt! is through the **RobotCommander** class.  
Thus we import moveit_commander.  

Initiante moveit_commander and a ROS node

In [ ]:
import sys
import copy
import rospy
import moveit_commander
import moveit_msgs.msg
import moveit_msgs.srv
import geometry_msgs.msg

print "============ STARTING TUTORIAL SETUP ============="
moveit_commander.roscpp_initialize(sys.argv)
rospy.init_node('move_group_node_04')

# Robot commander object -> interface the robot as a whole
robot = moveit_commander.RobotCommander()
# Planning scene object -> interface the world of the robot
scene = moveit_commander.PlanningSceneInterface()
# MoveGroup object
group = moveit_commander.MoveGroupCommander("right_arm")
right_eef = moveit_commander.MoveGroupCommander("right_eef")

# Publisher to display trajectories in Rviz.
rviz_pub = rospy.Publisher('/move_group/display_planned_path', moveit_msgs.msg.DisplayTrajectory, queue_size=10)
# Rviz must be opened manually:
# roslaunch golem_moveit_config demo.launch
print "============ Waiting for RVIZ... ============"
# rospy.sleep(5)
print "============= Starting tutorial =============\n"

============ STARTING TUTORIAL SETUP =============


## 2. Getting basic information

In [2]:
print "Reference frame: %s" % group.get_planning_frame()
print "Robot Groups:", robot.get_group_names()

Reference frame: /world
Robot Groups: ['chest', 'left_arm', 'left_eef', 'right_arm', 'right_eef']


In [6]:
print "======== ENTIRE ROBOT STATE ========"
robot.get_current_state()

======== ENTIRE ROBOT STATE ========


joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: /world
  name: ['mb', 'm0', 'm1', 'm2', 'm10', 'm11', 'm12', 'm13', 'm14', 'm15', 'm17', 'm20', 'm21', 'm22', 'm23', 'm24', 'm25', 'm27']
  position: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  velocity: []
  effort: []
multi_dof_joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: /world
  joint_names: []
  transforms: []
  twist: []
  wrench: []
attached_collision_objects: []
is_diff: False

## 3. Planning to a pose goal  
Plan a motion for this group to a desired pose for the end-effector.  

In [2]:
print "COMPUTING PLAN 1..."
# We use a standard pose msg
pose_target = geometry_msgs.msg.Pose()
pose_target.orientation.x = 0.705865
pose_target.orientation.y = 0.137701
pose_target.orientation.z = 0.687766
pose_target.orientation.w = -0.0988454

pose_target.position.x = 0.678809
pose_target.position.y = -0.139463
pose_target.position.z = 1.00571

# Send the pose to the planner
group.set_pose_target(pose_target)

COMPUTING PLAN 1...


In [3]:
r_arm_joints = (-0.837832, -1.12992, 1.967752, 0.625803, -0.065803, 0.0)

r_arm_joints = (0.5, -0.5, 0.0, 0.0, 0.0, 0.0)

print "============ Joint values: ", r_arm_joints
group.set_joint_value_target(r_arm_joints)

plan2 = group.plan()
group.go()

print "============ Waiting while RVIZ displays plan2..."
rospy.sleep(5)


============ Joint values:  (0.5, -0.5, 0.0, 0.0, 0.0, 0.0)
============ Waiting while RVIZ displays plan2...


In [3]:
group.set_planner_id('RRTkConfigDefault')
group.

In [8]:
# Call the planner to compute the plan
plan1 = group.plan()
print "Waiting while Rviz displays plan1..."
rospy.sleep(5)
group.go()

Waiting while Rviz displays plan1...


False

### Some useful properties  of moveit_commander.MoveGroupCommander()

- arm.get_current_joint_values()
- arm.get_current_pose()
- arm.get_current_rpy()

## 4. Kinematics

We can compute forward and inverse kinematics through services offered by moveit_commander:

- [/compute_ik](http://docs.ros.org/api/moveit_msgs/html/srv/GetPositionIK.html)
- [/compute_fk](http://docs.ros.org/api/moveit_msgs/html/srv/GetPositionFK.html)

### 4.1 Inverse kinematics service

In [ ]:
# Init service client object
rospy.wait_for_service('/golem_kinematics/right_arm/compute_ik')
print "IK service was found!"
compute_ik = rospy.ServiceProxy('/golem_kinematics/right_arm/compute_ik', moveit_msgs.srv.GetPositionIK)

In [ ]:
# srv request definition
reqik = moveit_msgs.srv.GetPositionIKRequest()

# Desired end effector position
desired_pose = geometry_msgs.msg.PoseStamped()
desired_pose.pose.position.x = 0.862551918502
desired_pose.pose.position.y = 0.00557360531441
desired_pose.pose.position.z = 1.55093489162
desired_pose.pose.orientation.x = 0.438226117241
desired_pose.pose.orientation.y = -0.445485839948
desired_pose.pose.orientation.z = 0.265900911042
desired_pose.pose.orientation.w = 0.734027889171

reqik.ik_request.group_name = 'right_arm'
reqik.ik_request.robot_state = robot.get_current_state()
# reqik.ik_request.constraints
reqik.ik_request.avoid_collisions = True
reqik.ik_request.ik_link_name = 'r_eef'
reqik.ik_request.pose_stamped = desired_pose
reqik.ik_request.timeout.secs = 5
reqik.ik_request.attempts = 0

In [ ]:
%time respik = compute_ik(reqik)
print respik.solution.joint_state.name
print respik.solution.joint_state.position
print 'Error code: ', respik.error_code.val

CPU times: user 0 ns, sys: 8 ms, total: 8 ms  
Wall time: 727 ms  
['m0', 'm10', 'm11', 'm12', 'm13', 'm14']  
(-0.49726, 1.72036, -0.9031, 1.54248, -0.89248, 0.0)  
Error code:  1  

### 4.2 Forward kinematics

In [ ]:
# Initialize service client
rospy.wait_for_service('/golem_kinematics/right_arm/compute_fk')
compute_fk = rospy.ServiceProxy('/golem_kinematics/right_arm/compute_fk', moveit_msgs.srv.GetPositionFK)

In [ ]:
# srv request definition
reqfk = moveit_msgs.srv.GetPositionFKRequest()
reqfk.robot_state = robot.get_current_state()

In [ ]:
compute_fk(reqfk)